<a href="https://colab.research.google.com/github/AntonioBorrego/Spark/blob/main/AnalisisBases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!pip install pyspark

In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
import pandas as pd
import networkx as net
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 
from pyspark.sql.functions import *
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [ ]:
# Creamos una función udf para transformar los True en "Optimo" y los False en "No Optimo".
def rend(rendimiento):
  if rendimiento==True:
    return "Optimo"
  if rendimiento==False:
    return "No Optimo"

usoUdf = udf(rend)

# JULIO

In [ ]:
opt_jul = spark.read.load("/content/optimo_julio.csv",'com.databricks.spark.csv',header='true',inferSchema='true')
opt_jul.show()

+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|        Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|  0|         24|     135|     73|      71|  144|1.8088179876899886|             -2|     Optimo|
|  1|         27|      43|     78|      61|  139| 1.746011807561864|            -17|  No Optimo|
|  2|         27|      90|     69|      54|  123|1.5450320311518653|            -15|  No Optimo|
|  3|         24|     129|     60|      52|  112| 1.406858434869991|             -8|     Optimo|
|  4|         24|      57|     58|      53|  111|1.3942971988443662|             -5|     Optimo|
|  5|         24|     132|     54|      54|  108|1.3566134907674914|              0|     Optimo|
|  6|         24|     163|     50|      50|  100|1.2561236025624922|              0|     Optimo|
|  7|         24|      19|    

In [ ]:
no_optimas = opt_jul.filter(opt_jul.Rendimiento == "No Optimo")

In [ ]:
no_optimas_rdd=no_optimas.rdd.map(tuple)

In [ ]:
estaciones_n_o=no_optimas_rdd.map(lambda x: x[2]).collect()

In [ ]:
estaciones_n_o

[43,
 90,
 74,
 17,
 25,
 12,
 21,
 62,
 8,
 145,
 99,
 65,
 148,
 119,
 153,
 142,
 97,
 130,
 94,
 143,
 158,
 146,
 35,
 115,
 120,
 100,
 39,
 150,
 138,
 69,
 15,
 152,
 151,
 144,
 104,
 92,
 112,
 101,
 140,
 29,
 107,
 105,
 141,
 88,
 28,
 173,
 147,
 72,
 159,
 32,
 20,
 61]

In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones Julio.
opt_jul.createOrReplaceTempView("filtrado")
booleano_jul_total= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_jul_total = booleano_jul_total.toPandas()
booleano_jul_total.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|      52|
|     Optimo|     118|
+-----------+--------+



In [ ]:
no_optimas.createOrReplaceTempView("filtrado")
booleano = spark.sql("select Rendimiento, count(rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda = booleano.toPandas()
booleano.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|      52|
+-----------+--------+



# AGOSTO

In [ ]:
opt_ago = spark.read.load("/content/optimos_Agosto.csv",'com.databricks.spark.csv',header='true',inferSchema='true')
opt_ago.show()

+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|        Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|  0|         27|      43|    541|     565| 1106|1.9915010083549411|             24|  No Optimo|
|  1|         24|     163|    400|     401|  801|1.4423076923076923|              1|     Optimo|
|  2|         24|     149|    383|     384|  767|1.3810861423220975|              1|     Optimo|
|  3|         24|      57|    376|     386|  762|1.3720829732065687|             10|     Optimo|
|  4|         24|     135|    376|     357|  733|1.3198645923365024|            -19|  No Optimo|
|  5|         24|     129|    355|     357|  712| 1.282051282051282|              2|     Optimo|
|  6|         27|      90|    342|     339|  681|1.2262316335350043|             -3|     Optimo|
|  7|         24|       1|    

In [ ]:
comp_ago = opt_ago[opt_ago['estacion'].isin(estaciones_n_o)]
comp_ago.show()

+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|         Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+
|  0|         27|      43|    541|     565| 1106| 1.9915010083549411|             24|  No Optimo|
|  6|         27|      90|    342|     339|  681| 1.2262316335350043|             -3|     Optimo|
| 40|         24|      74|    213|     207|  420|  0.756266205704408|             -6|     Optimo|
| 50|         24|     145|    188|     195|  383| 0.6896427542494958|              7|     Optimo|
| 52|         24|     115|    190|     183|  373| 0.6716364160184385|             -7|     Optimo|
| 55|         30|      21|    190|     176|  366| 0.6590319792566983|            -14|     Optimo|
| 56|         24|      62|    181|     184|  365| 0.6572313454335926|              3|     Optimo|
| 59|         21|   

In [ ]:
mejoras_ago = comp_ago.withColumn('Nuevas_Total_Bases',(comp_ago.total_bases + (comp_ago.Demanda_Deficit/abs(comp_ago.Demanda_Deficit))*round(comp_ago.total_bases*0.3)))
mejoras_ago = mejoras_ago.withColumn("Nuevo_Rendimiento", ((mejoras_ago.Demanda_Deficit>=(-(mejoras_ago.Nuevas_Total_Bases)/2)) & (mejoras_ago.Demanda_Deficit<=((mejoras_ago.Nuevas_Total_Bases)/2)) & (mejoras_ago.Flujo > mejoras_ago.Nuevas_Total_Bases)))
mejoras_ago = mejoras_ago.withColumn("Nuevo_Rendimiento", usoUdf(mejoras_ago.Nuevo_Rendimiento))
mejoras_ago.show()

+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+------------------+-----------------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|         Frecuencia|Demanda_Deficit|Rendimiento|Nuevas_Total_Bases|Nuevo_Rendimiento|
+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+------------------+-----------------+
|  0|         27|      43|    541|     565| 1106| 1.9915010083549411|             24|  No Optimo|              35.0|        No Optimo|
|  6|         27|      90|    342|     339|  681| 1.2262316335350043|             -3|     Optimo|              19.0|           Optimo|
| 40|         24|      74|    213|     207|  420|  0.756266205704408|             -6|     Optimo|              17.0|           Optimo|
| 50|         24|     145|    188|     195|  383| 0.6896427542494958|              7|     Optimo|              31.0|           Optimo|
| 52|         24|     115|    190|     183|  373| 0.671

In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones de Agosto.
opt_ago.createOrReplaceTempView("filtrado")
booleano_ago_total= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_ago_total = booleano_ago_total.toPandas()
booleano_ago_total.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|      22|
|     Optimo|     147|
+-----------+--------+



In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones no optimas de Julio en Agosto.
comp_ago.createOrReplaceTempView("filtrado")
booleano_ago= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_ago = booleano_ago.toPandas()
booleano_ago.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|       4|
|     Optimo|      49|
+-----------+--------+



In [ ]:
# Hacemos un nuevo recuento para ver como evolucionan las estaciones después de aplicar el criterio de mejora
mejoras_ago.createOrReplaceTempView("filtrado")
booleano_mej_ago = spark.sql("select nuevo_Rendimiento, count(nuevo_Rendimiento) as Cantidad from filtrado group by nuevo_Rendimiento")
booleano_panda_mej_ago = booleano_mej_ago.toPandas()
booleano_mej_ago.show()

+-----------------+--------+
|nuevo_Rendimiento|Cantidad|
+-----------------+--------+
|             null|       0|
|        No Optimo|       8|
|           Optimo|      44|
+-----------------+--------+



In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=booleano_panda_ago.Rendimiento,
                y=booleano_panda_ago.Cantidad,
                name='Antes de Mejorar',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_mej_ago.nuevo_Rendimiento,
                y=booleano_panda_mej_ago.Cantidad,
                name='Después de Mejorar',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Agosto',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
fig.show()

# SEPTIEMBRE


In [ ]:
opt_sep = spark.read.load("/content/optimo_septiembre.csv",'com.databricks.spark.csv',header='true',inferSchema='true')
opt_sep.show()

+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|        Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|  0|         24|      57|    133|     144|  277|1.8173468048812491|             11|     Optimo|
|  1|         24|     135|    126|     117|  243|1.5942789660149588|             -9|     Optimo|
|  2|         24|       1|    115|     112|  227|1.4893058653719986|             -3|     Optimo|
|  3|         27|      90|    120|     106|  226|1.4827450465818135|            -14|  No Optimo|
|  4|         24|      64|    110|     113|  223|1.4630625902112584|              3|     Optimo|
|  5|         24|     149|     98|     116|  214|1.4040152210995933|             18|  No Optimo|
|  6|         21|      58|     98|     102|  200| 1.312163758037003|              4|     Optimo|
|  7|         24|     163|    

In [ ]:
comp_sep = opt_sep[opt_sep['estacion'].isin(estaciones_n_o)]
comp_sep.show()

+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|         Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+
|  3|         27|      90|    120|     106|  226| 1.4827450465818135|            -14|  No Optimo|
|  8|         27|      43|     97|      93|  190|  1.246555570135153|             -4|     Optimo|
| 17|         24|      62|     76|      84|  160| 1.0497310064296024|              8|     Optimo|
| 38|         24|      25|     68|      56|  124| 0.8135415299829418|            -12|     Optimo|
| 39|         24|      74|     65|      58|  123| 0.8069807111927568|             -7|     Optimo|
| 61|         21|       8|     46|      47|   93| 0.6101561474872064|              1|     Optimo|
| 62|         24|       8|     46|      47|   93| 0.6101561474872064|              1|     Optimo|
| 64|         24|   

In [ ]:
mejoras_sep = comp_sep.withColumn('Nuevas_Total_Bases',(comp_sep.total_bases + (comp_sep.Demanda_Deficit/abs(comp_sep.Demanda_Deficit))*round(comp_sep.total_bases*0.3)))
mejoras_sep = mejoras_sep.withColumn("Nuevo_Rendimiento", ((mejoras_sep.Demanda_Deficit>=(-(mejoras_sep.Nuevas_Total_Bases)/2)) & (mejoras_sep.Demanda_Deficit<=((mejoras_sep.Nuevas_Total_Bases)/2)) & (mejoras_sep.Flujo > mejoras_sep.Nuevas_Total_Bases)))
mejoras_sep = mejoras_sep.withColumn("Nuevo_Rendimiento", usoUdf(mejoras_sep.Nuevo_Rendimiento))
mejoras_sep.show()

+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+------------------+-----------------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|         Frecuencia|Demanda_Deficit|Rendimiento|Nuevas_Total_Bases|Nuevo_Rendimiento|
+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+------------------+-----------------+
|  3|         27|      90|    120|     106|  226| 1.4827450465818135|            -14|  No Optimo|              19.0|        No Optimo|
|  8|         27|      43|     97|      93|  190|  1.246555570135153|             -4|     Optimo|              19.0|           Optimo|
| 17|         24|      62|     76|      84|  160| 1.0497310064296024|              8|     Optimo|              31.0|           Optimo|
| 38|         24|      25|     68|      56|  124| 0.8135415299829418|            -12|     Optimo|              17.0|        No Optimo|
| 39|         24|      74|     65|      58|  123| 0.806

In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones de Septiembre.
opt_sep.createOrReplaceTempView("filtrado")
booleano_sep_total= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_sep_total = booleano_sep_total.toPandas()
booleano_sep_total.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|      20|
|     Optimo|     148|
+-----------+--------+



In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones no optimas de julio en Septiembre.
comp_sep.createOrReplaceTempView("filtrado")
booleano_sep= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_sep = booleano_sep.toPandas()
booleano_sep.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|       9|
|     Optimo|      44|
+-----------+--------+



In [ ]:
# Hacemos un nuevo recuento para ver como evolucionan las estaciones  después de aplicar el criterio de mejora
mejoras_sep.createOrReplaceTempView("filtrado")
booleano_mej_sep = spark.sql("select nuevo_Rendimiento, count(nuevo_Rendimiento) as Cantidad from filtrado group by nuevo_Rendimiento")
booleano_panda_mej_sep = booleano_mej_sep.toPandas()
booleano_mej_sep.show()

+-----------------+--------+
|nuevo_Rendimiento|Cantidad|
+-----------------+--------+
|             null|       0|
|        No Optimo|      15|
|           Optimo|      32|
+-----------------+--------+



In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=booleano_panda_sep.Rendimiento,
                y=booleano_panda_sep.Cantidad,
                name='Antes de Mejorar',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_mej_sep.nuevo_Rendimiento,
                y=booleano_panda_mej_sep.Cantidad,
                name='Después de Mejorar',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Septiembre',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
fig.show()

# OCTUBRE

In [ ]:
opt_oct = spark.read.load("/content/optimo_octubre.csv",'com.databricks.spark.csv',header='true',inferSchema='true')
opt_oct.show()

+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|        Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|  0|         24|      64|     97|     101|  198|1.4772812057002163|              4|     Optimo|
|  1|         24|     129|     87|      96|  183|1.3653659628441392|              9|     Optimo|
|  2|         24|     163|     87|      86|  173|1.2907558009400881|             -1|     Optimo|
|  3|         24|     162|     85|      83|  168|1.2534507199880625|             -2|     Optimo|
|  4|         24|     160|     69|      87|  156|1.1639185257032008|             18|  No Optimo|
|  5|         24|     149|     75|      77|  152|1.1340744609415803|              2|     Optimo|
|  6|         24|       9|     82|      66|  148|1.1042303961799596|            -16|  No Optimo|
|  7|         24|     135|    

In [ ]:
comp_oct = opt_oct[opt_oct['estacion'].isin(estaciones_n_o)]
comp_oct.show()

+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|        Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|  8|         27|      90|     78|      65|  143| 1.066925315227934|            -13|     Optimo|
| 13|         27|      43|     63|      70|  133|0.9923151533238828|              7|     Optimo|
| 22|         24|     130|     63|      55|  118|0.8803999104678057|             -8|     Optimo|
| 26|         30|      21|     70|      45|  115|0.8580168618965903|            -25|  No Optimo|
| 33|         24|      74|     50|      59|  109|0.8132507647541596|              9|     Optimo|
| 38|         24|     115|     59|      45|  104|0.7759456838021339|            -14|  No Optimo|
| 51|         24|     145|     51|      42|   93|0.6938745057076774|             -9|     Optimo|
| 52|         24|      62|    

In [ ]:
mejoras_oct = comp_oct.withColumn('Nuevas_Total_Bases',(comp_oct.total_bases + (comp_oct.Demanda_Deficit/abs(comp_oct.Demanda_Deficit))*round(comp_oct.total_bases*0.3)))
mejoras_oct = mejoras_oct.withColumn("Nuevo_Rendimiento", ((mejoras_oct.Demanda_Deficit>=(-(mejoras_oct.Nuevas_Total_Bases)/2)) & (mejoras_oct.Demanda_Deficit<=((mejoras_oct.Nuevas_Total_Bases)/2)) & (mejoras_oct.Flujo > mejoras_oct.Nuevas_Total_Bases)))
mejoras_oct = mejoras_oct.withColumn("Nuevo_Rendimiento", usoUdf(mejoras_oct.Nuevo_Rendimiento))
columnas = ["total_bases", "Frecuencia", "Salidas", "Llegadas", "Rendimiento"]
mejoras_oct = mejoras_oct.drop(*columnas)
mejoras_oct.show()

+---+--------+-----+---------------+------------------+-----------------+
|_c0|estacion|Flujo|Demanda_Deficit|Nuevas_Total_Bases|Nuevo_Rendimiento|
+---+--------+-----+---------------+------------------+-----------------+
|  8|      90|  143|            -13|              19.0|        No Optimo|
| 13|      43|  133|              7|              35.0|           Optimo|
| 22|     130|  118|             -8|              17.0|           Optimo|
| 26|      21|  115|            -25|              21.0|        No Optimo|
| 33|      74|  109|              9|              31.0|           Optimo|
| 38|     115|  104|            -14|              17.0|        No Optimo|
| 51|     145|   93|             -9|              17.0|        No Optimo|
| 52|      62|   92|              8|              31.0|           Optimo|
| 53|      65|   91|             -9|              17.0|        No Optimo|
| 72|       8|   80|              6|              27.0|           Optimo|
| 73|       8|   80|              6|  

In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones Octubre.
opt_oct.createOrReplaceTempView("filtrado")
booleano_oct_total= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_oct_total = booleano_oct_total.toPandas()
booleano_oct_total.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|      20|
|     Optimo|     147|
+-----------+--------+



In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones no optimas de Julio en Octubre.
comp_oct.createOrReplaceTempView("filtrado")
booleano_oct= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_oct = booleano_oct.toPandas()
booleano_oct.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|       5|
|     Optimo|      48|
+-----------+--------+



In [ ]:
# Hacemos un nuevo recuento para ver como evolucionan las estaciones no óptimas después de aplicar el criterio de mejora
mejoras_oct.createOrReplaceTempView("filtrado")
booleano_mej_oct = spark.sql("select nuevo_Rendimiento, count(nuevo_Rendimiento) as Cantidad from filtrado group by nuevo_Rendimiento")
booleano_panda_mej_oct = booleano_mej_oct.toPandas()
booleano_mej_oct.show()

+-----------------+--------+
|nuevo_Rendimiento|Cantidad|
+-----------------+--------+
|             null|       0|
|        No Optimo|      13|
|           Optimo|      37|
+-----------------+--------+



In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=booleano_panda_oct.Rendimiento,
                y=booleano_panda_oct.Cantidad,
                name='Antes de Mejorar',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_mej_oct.nuevo_Rendimiento,
                y=booleano_panda_mej_oct.Cantidad,
                name='Después de Mejorar',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Octubre',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
fig.show()

# NOVIEMBRE


In [ ]:
opt_nov = spark.read.load("/content/optimos_Noviembre.csv",'com.databricks.spark.csv',header='true',inferSchema='true')
opt_nov.show()

+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|        Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|  0|         24|     135|    293|     275|  568| 1.569147466710868|            -18|  No Optimo|
|  1|         24|     163|    281|     286|  567| 1.566384883142715|              5|     Optimo|
|  2|         24|      57|    271|     276|  547|1.5111332117796563|              5|     Optimo|
|  3|         27|      43|    261|     258|  519|1.4337808718713743|             -3|     Optimo|
|  4|         24|     149|    232|     254|  486|1.3426156141223273|             22|  No Optimo|
|  5|         24|     168|    239|     246|  485|1.3398530305541743|              7|     Optimo|
|  6|         24|     129|    238|     231|  469|1.2956516934637274|             -7|     Optimo|
|  7|         27|      90|    

In [ ]:
comp_nov = opt_nov[opt_nov['estacion'].isin(estaciones_n_o)]
comp_nov.show()

+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|         Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+
|  3|         27|      43|    261|     258|  519| 1.4337808718713743|             -3|     Optimo|
|  7|         27|      90|    232|     230|  462| 1.2763136084866566|             -2|     Optimo|
| 21|         24|      62|    165|     176|  341| 0.9420409967401514|             11|     Optimo|
| 41|         24|      74|    149|     125|  274| 0.7569478976739047|            -24|  No Optimo|
| 62|         24|     145|    111|     110|  221|  0.610530968561799|             -1|     Optimo|
| 65|         24|      65|    107|     106|  213| 0.5884303000165755|             -1|     Optimo|
| 68|         24|     115|    105|     105|  210| 0.5801425493121167|              0|     Optimo|
| 71|         24|   

In [ ]:
mejoras_nov = comp_nov.withColumn('Nuevas_Total_Bases',(comp_nov.total_bases + (comp_nov.Demanda_Deficit/abs(comp_nov.Demanda_Deficit))*round(comp_nov.total_bases*0.3)))
mejoras_nov = mejoras_nov.withColumn("Nuevo_Rendimiento", ((mejoras_nov.Demanda_Deficit>=(-(mejoras_nov.Nuevas_Total_Bases)/2)) & (mejoras_nov.Demanda_Deficit<=((mejoras_nov.Nuevas_Total_Bases)/2)) & (mejoras_nov.Flujo > mejoras_nov.Nuevas_Total_Bases)))
mejoras_nov = mejoras_nov.withColumn("Nuevo_Rendimiento", usoUdf(mejoras_nov.Nuevo_Rendimiento))
columnas = ["total_bases", "Frecuencia", "Salidas", "Llegadas", "Rendimiento"]
mejoras_nov = mejoras_nov.drop(*columnas)
mejoras_nov.show()

+---+--------+-----+---------------+------------------+-----------------+
|_c0|estacion|Flujo|Demanda_Deficit|Nuevas_Total_Bases|Nuevo_Rendimiento|
+---+--------+-----+---------------+------------------+-----------------+
|  3|      43|  519|             -3|              19.0|           Optimo|
|  7|      90|  462|             -2|              19.0|           Optimo|
| 21|      62|  341|             11|              31.0|           Optimo|
| 41|      74|  274|            -24|              17.0|        No Optimo|
| 62|     145|  221|             -1|              17.0|           Optimo|
| 65|      65|  213|             -1|              17.0|           Optimo|
| 68|     115|  210|              0|              null|             null|
| 71|     130|  204|             -4|              17.0|           Optimo|
| 72|      12|  204|              8|              31.0|           Optimo|
| 74|       8|  202|             12|              27.0|           Optimo|
| 75|       8|  202|             12|  

In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones Noviembre.
opt_nov.createOrReplaceTempView("filtrado")
booleano_nov_total= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_nov_total = booleano_nov_total.toPandas()
booleano_nov_total.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|      24|
|     Optimo|     144|
+-----------+--------+



In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones no optimas de Julio en Noviembre.
comp_nov.createOrReplaceTempView("filtrado")
booleano_nov= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_nov = booleano_nov.toPandas()
booleano_nov.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|       5|
|     Optimo|      48|
+-----------+--------+



In [ ]:
# Hacemos un nuevo recuento para ver como evolucionan las estaciones no óptimas después de aplicar el criterio de mejora
mejoras_nov.createOrReplaceTempView("filtrado")
booleano_mej_nov = spark.sql("select nuevo_Rendimiento, count(nuevo_Rendimiento) as Cantidad from filtrado group by nuevo_Rendimiento")
booleano_panda_mej_nov = booleano_mej_nov.toPandas()
booleano_mej_nov.show()

+-----------------+--------+
|nuevo_Rendimiento|Cantidad|
+-----------------+--------+
|             null|       0|
|        No Optimo|      11|
|           Optimo|      39|
+-----------------+--------+



In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=booleano_panda_nov.Rendimiento,
                y=booleano_panda_nov.Cantidad,
                name='Antes de Mejorar',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_mej_nov.nuevo_Rendimiento,
                y=booleano_panda_mej_nov.Cantidad,
                name='Después de Mejorar',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Noviembre',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
fig.show()

# DICIEMBRE


In [ ]:
opt_dic = spark.read.load("/content/optimos_Diciembre.csv",'com.databricks.spark.csv',header='true',inferSchema='true')
opt_dic.show()

+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|        Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+------------------+---------------+-----------+
|  0|         27|      43|    828|     802| 1630|1.4250867729216028|            -26|  No Optimo|
|  1|         24|     163|    798|     818| 1616|1.4128467638290245|             20|  No Optimo|
|  2|         24|     129|    807|     804| 1611|1.4084753320102466|             -3|     Optimo|
|  3|         24|     135|    800|     766| 1566|1.3691324456412453|            -34|  No Optimo|
|  4|         24|      57|    753|     765| 1518|1.3271667001809773|             12|     Optimo|
|  5|         24|     149|    752|     747| 1499| 1.310555259269621|             -5|     Optimo|
|  6|         27|      90|    703|     690| 1393| 1.217880904711529|            -13|     Optimo|
|  7|         24|     168|    

In [ ]:
comp_dic = opt_dic[opt_dic['estacion'].isin(estaciones_n_o)]
comp_dic.show()

+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+
|_c0|total_bases|estacion|Salidas|Llegadas|Flujo|         Frecuencia|Demanda_Deficit|Rendimiento|
+---+-----------+--------+-------+--------+-----+-------------------+---------------+-----------+
|  0|         27|      43|    828|     802| 1630| 1.4250867729216028|            -26|  No Optimo|
|  6|         27|      90|    703|     690| 1393|  1.217880904711529|            -13|     Optimo|
| 37|         24|      62|    430|     436|  866| 0.7571319910123362|              6|     Optimo|
| 55|         24|      74|    385|     379|  764| 0.6679547819092665|             -6|     Optimo|
| 56|         24|     130|    365|     390|  755| 0.6600862046354663|             25|  No Optimo|
| 58|         24|      65|    375|     373|  748| 0.6539662000891772|             -2|     Optimo|
| 61|         24|     145|    360|     369|  729| 0.6373547591778211|              9|     Optimo|
| 62|         21|   

In [ ]:
mejoras_dic = comp_dic.withColumn('Nuevas_Total_Bases',(comp_dic.total_bases + (comp_dic.Demanda_Deficit/abs(comp_dic.Demanda_Deficit))*round(comp_dic.total_bases*0.3)))
mejoras_dic = mejoras_dic.withColumn("Nuevo_Rendimiento", ((mejoras_dic.Demanda_Deficit>=(-(mejoras_dic.Nuevas_Total_Bases)/2)) & (mejoras_dic.Demanda_Deficit<=((mejoras_dic.Nuevas_Total_Bases)/2)) & (mejoras_dic.Flujo > mejoras_dic.Nuevas_Total_Bases)))
mejoras_dic = mejoras_dic.withColumn("Nuevo_Rendimiento", usoUdf(mejoras_dic.Nuevo_Rendimiento))
columnas = ["total_bases", "Frecuencia", "Salidas", "Llegadas", "Rendimiento"]
mejoras_dic = mejoras_dic.drop(*columnas)
mejoras_dic.show()

+---+--------+-----+---------------+------------------+-----------------+
|_c0|estacion|Flujo|Demanda_Deficit|Nuevas_Total_Bases|Nuevo_Rendimiento|
+---+--------+-----+---------------+------------------+-----------------+
|  0|      43| 1630|            -26|              19.0|        No Optimo|
|  6|      90| 1393|            -13|              19.0|        No Optimo|
| 37|      62|  866|              6|              31.0|           Optimo|
| 55|      74|  764|             -6|              17.0|           Optimo|
| 56|     130|  755|             25|              31.0|        No Optimo|
| 58|      65|  748|             -2|              17.0|           Optimo|
| 61|     145|  729|              9|              31.0|           Optimo|
| 62|       8|  727|              5|              27.0|           Optimo|
| 63|       8|  727|              5|              31.0|           Optimo|
| 70|      17|  669|             -1|              15.0|           Optimo|
| 74|     115|  656|            -16|  

In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones Diciembre.
opt_dic.createOrReplaceTempView("filtrado")
booleano_dic_total= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_dic_total = booleano_dic_total.toPandas()
booleano_dic_total.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|      43|
|     Optimo|     125|
+-----------+--------+



In [ ]:
# Recuento de estaciones optimas y no optimas de las estaciones no optimas de Julio en Diciembre.
comp_dic.createOrReplaceTempView("filtrado")
booleano_dic= spark.sql("select Rendimiento, count(Rendimiento) as Cantidad from filtrado group by Rendimiento")
booleano_panda_dic = booleano_dic.toPandas()
booleano_dic.show()

+-----------+--------+
|Rendimiento|Cantidad|
+-----------+--------+
|  No Optimo|      14|
|     Optimo|      39|
+-----------+--------+



In [ ]:
# Hacemos un nuevo recuento para ver como evolucionan las estaciones no óptimas después de aplicar el criterio de mejora
mejoras_dic.createOrReplaceTempView("filtrado")
booleano_mej_dic = spark.sql("select nuevo_Rendimiento, count(nuevo_Rendimiento) as Cantidad from filtrado group by nuevo_Rendimiento")
booleano_panda_mej_dic = booleano_mej_dic.toPandas()
booleano_mej_dic.show()

+-----------------+--------+
|nuevo_Rendimiento|Cantidad|
+-----------------+--------+
|             null|       0|
|        No Optimo|      15|
|           Optimo|      35|
+-----------------+--------+



In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=booleano_panda_dic.Rendimiento,
                y=booleano_panda_dic.Cantidad,
                name='Antes de Mejorar',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_mej_dic.nuevo_Rendimiento,
                y=booleano_panda_mej_dic.Cantidad,
                name='Después de Mejorar',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Diciembre',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
fig.show()

# COMPARACIÓN DE LOS DATOS

In [ ]:
# Hacemos un histograma con las estaciones no óptimas de Julio en los demás meses
fig = go.Figure()
fig.add_trace(go.Bar(x=booleano_panda_ago.Rendimiento,
                y=booleano_panda_ago.Cantidad,
                name='AGOSTO',
                marker_color='rgb(39,64,139)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_sep.Rendimiento,
                y=booleano_panda_sep.Cantidad,
                name='SEPTIEMBRE',
                marker_color='rgb(26, 118, 255)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_oct.Rendimiento,
                y=booleano_panda_oct.Cantidad,
                name='OCTUBRE',
                marker_color='rgb(108,166,205)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_nov.Rendimiento,
                y=booleano_panda_nov.Cantidad,
                name='NOVIEMBRE',
                marker_color='rgb(99,184,255)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_dic.Rendimiento,
                y=booleano_panda_dic.Cantidad,
                name='DICIEMBRE',
                marker_color='rgb(135,206,255)'
                ))
fig.update_layout(
    title='Comparación por Meses',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
fig.show()

In [ ]:
# Hacemos un histograma de las estaciones de Julio en los demás meses después de aplicar el criterio de mejora
fig = go.Figure()
fig.add_trace(go.Bar(x=booleano_panda_mej_ago.nuevo_Rendimiento,
                y=booleano_panda_mej_ago.Cantidad,
                name='AGOSTO',
                marker_color='rgb(39,64,139)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_mej_sep.nuevo_Rendimiento,
                y=booleano_panda_mej_sep.Cantidad,
                name='SEPTIEMBRE',
                marker_color='rgb(26, 118, 255)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_mej_oct.nuevo_Rendimiento,
                y=booleano_panda_mej_oct.Cantidad,
                name='OCTUBRE',
                marker_color='rgb(108,166,205)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_mej_nov.nuevo_Rendimiento,
                y=booleano_panda_mej_nov.Cantidad,
                name='NOVIEMBRE',
                marker_color='rgb(99,184,255)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_mej_dic.nuevo_Rendimiento,
                y=booleano_panda_mej_dic.Cantidad,
                name='DICIEMBRE',
                marker_color='rgb(135,206,255)'
                ))
fig.update_layout(
    title='Comparación por Meses aplicada mejora',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
fig.show()

In [ ]:
# Hacemos un histograma con las estaciones óptimas y no óptimas de cada mes
fig = go.Figure()
fig.add_trace(go.Bar(x=booleano_panda_jul_total.Rendimiento,
                y=booleano_panda_jul_total.Cantidad,
                name='JULIO',
                marker_color='rgb(0,0,128)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_ago_total.Rendimiento,
                y=booleano_panda_ago_total.Cantidad,
                name='AGOSTO',
                marker_color='rgb(39,64,139)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_sep_total.Rendimiento,
                y=booleano_panda_sep_total.Cantidad,
                name='SEPTIEMBRE',
                marker_color='rgb(26, 118, 255)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_oct_total.Rendimiento,
                y=booleano_panda_oct_total.Cantidad,
                name='OCTUBRE',
                marker_color='rgb(108,166,205)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_nov_total.Rendimiento,
                y=booleano_panda_nov_total.Cantidad,
                name='NOVIEMBRE',
                marker_color='rgb(99,184,255)'
                ))
fig.add_trace(go.Bar(x=booleano_panda_dic_total.Rendimiento,
                y=booleano_panda_dic_total.Cantidad,
                name='DICIEMBRE',
                marker_color='rgb(135,206,255)'
                ))
fig.update_layout(
    title='Rendimiento por meses',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Cantidad',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
fig.show()